# EDA

Check dataset

In [3]:
import torch
import numpy
from torchvision import datasets, models
from torch.utils.data import Dataset, DataLoader
from random import randint
from PIL import Image, ImageDraw
from transforms import data_transforms
import logging
import os
import csv
import random
import json
import pandas

In [4]:
timeclasses = json.load(open('timeclasses.json'))

def find_class_id(year):
  original_year = year
  year = year.replace('c.', '')
  lbl = ''
  for el in year:
    if el not in '0123456789.':
        continue
    lbl = lbl + el
  try:
      year = int(float(lbl))
  except:
      print(year)
      return None
  for i in range(len(timeclasses)):
    start, end = timeclasses[i].split('_')
    start = int(start)
    end = int(end)
    if start <= year <= end:
      if i == 0:
          logging.debug(f"{original_year} -> {year}")
      return i
  logging.warning(f"{year} is not in classification")
  return None

In [3]:
df = pandas.read_csv('../data/artdataset/data.csv', engine='python')
df = df.dropna(axis=0)
df

,filename,artist,title,style,genre,date
0,102257.jpg,5b39c876740bfc1cfaf544721c43cac3,Uriel,Color Field Painting,abstract,1955.0
1,75232.jpg,5b39c876740bfc1cfaf544721c43cac3,Vir Heroicus Sublimis,Color Field Painting,abstract,1950.0
4,63861.jpg,5b39c876740bfc1cfaf544721c43cac3,Black Fire I,Color Field Painting,abstract,1963.0
5,53180.jpg,782192dc825c7c1149825adcb7caacbc,Camellia and Bush Warbler,Ukiyo-e,bird-and-flower painting,1844
6,70944.jpg,afdfc43410410892bc9d1c4d3c3d6220,Forbidden Land,Surrealism,landscape,1937.0
...,...,...,...,...,...,...
79424,9400.jpg,ece3c2c2f8cd7d618f008715c70d2d68,Tulip,Abstract Expressionism,abstract,1978.0
79426,17971.jpg,e4f90e4cae5e75ee90d8852c013c0ff2,Study of Horse Heads,Early Renaissance,sketch and study,1433
79427,36690.jpg,d9dec036e03403f4c1b795ba026925d1,Structures and Signs,Abstract Art,abstract,1957.0
79429,25525.jpg,2d72f2000c42051e7c350a39bdce9bc1,Number 13A (Arabesque),Action painting,abstract,1948.0


In [4]:
cnt_dates = {}
for el in df['date']:
    if el not in cnt_dates:
        cnt_dates[el] = 0
    cnt_dates[el] += 1
for el in cnt_dates:
    print(el, cnt_dates[el])

1955.0 170
1950.0 187
1963.0 151
1844 52
1937.0 130
1873 229
1515 59
1850 61
1838 38
1952.0 107
1439 3
1896 283
1824 48
1866 133
1612.0 5
1835 62
c.1645 28
1936.0 121
1635 66
1534 9
1924.0 191
1783 25
1925.0 170
1541 7
1926.0 185
1905.0 165
1910 418
1476 3
1958.0 138
1967.0 144
1839 44
1768 9
1672 8
1884 303
c.1662 4
1648 30
1547 5
1895 317
1934 308
1505 34
c.1757 2
1559 10
1600 11
c.1910 138
1899 263
c.1531 5
c.1630 39
1892 271
1880 277
1879 226
c.1896 35
1915 342
1882 299
1523 24
1826 53
c.1749 2
c.1895 155
1906 302
1645 35
c.1669 4
1898 255
1856 59
1891 256
1904 251
1897 242
1949.0 109
1913 440
1869 145
c.1660 18
c.1906 43
1916 291
c.1902 95
1901 229
1911 349
c.1907 70
1848 58
1912 439
1919 268
c.1916 28
1921 201
1699.0 5
1909 326
c.1912 69
1914 404
1907 349
1887.0 72
c.1914 34
1905 284
1890 346
1843 63
1707.0 28
1918 281
1894 279
c.1878 59
1883 270
1875 210
1878 205
1890.0 129
c.1897 59
c.1923 61
c.1915 76
1623 22
1907.0 168
1900 281
1908 388
c.1803 2
1888 346
c.1880 146
1876 200
1

1573.0 4
c.1617 4
c.1742 4
c.1477 1
1556.0 1
2006.0 12
1682 4
c.1541 4
c.1335 2
c.1766 2
c.1422 2
c.1100 3
1818.0 11
1656.0 14
1514.0 1
c.1551 3
1676.0 5
1757.0 5
c.1858 9
1652.0 2
c.1859 7
c.1513 5
c.1483 3
c.1449 2
1705.0 19
c.1471 4
1821.0 16
1768.0 2
1734 3
c.1482 2
1720.0 5
1779.0 5
1641.0 2
c.1747 5
1831.0 13
1739.0 2
c.1641 1
1588.0 2
1731.0 6
1328 1
1646.0 3
1633.0 2
1728.0 2
1809.0 16
1592.0 2
c.1946 1
1424 4
1717 4
c.1537 6
1744.0 2
1616.0 1
June 1956 1
1535.0 3
1637.0 2
1674 3
1490.0 2
1501 1
c.1970 4
1668.0 2
c.1313 1
c.1768 7
1629.0 1
1632.0 2
c.1786 3
1708 2
1429 4
c.1606 4
1761.0 3
904.0 1
1528.0 2
c.1829 3
c.1609 4
1741.0 1
1527.0 3
1745.0 6
1205 1
c.1941 6
1727 3
1610.0 6
1592 5
c.1841 4
1598.0 3
1639.0 1
1486.0 6
c.1573 2
1690 3
c.1798 7
c.1334 1
1752.0 2
1754.0 1
1769.0 10
1624.0 3
c.1969 3
1815.0 24
1603.0 1
1777.0 7
1589.0 1
1724.0 1
1642.0 4
1843.0 18
1654.0 1
1523.0 2
c.1474 2
c.1463 1
c.1754 2
c.1977 3
c.1412 1
c.1466 1
1320 1
1463 1
c.1936 6
1767.0 3
1596.0 3
c

In [5]:
def class_id2label(timeid):
    return timeclasses[timeid]
class_id2label(3)

'1601_1751'

In [6]:
df['timeid'] = df['date'].apply(find_class_id)
df = df.dropna(axis=0)
df.hist(column="timeid")[0]

array([<matplotlib.axes._subplots.AxesSubplot object at 0x7f3770b9e588>],
      dtype=object)

In [7]:
to_save = df[['filename', 'timeid']]
to_save.to_csv('../data/artdataset/info.csv', index=False)
to_save

NameError: name 'df' is not defined

In [8]:
sum(df['timeid'] == 0)

274

In [1]:
from transforms import data_transforms
from datasets import TimeDataset, MiniDataset, FeatureDataset
import pickle
import importlib

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
)
Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
)


In [3]:
from transforms import data_transforms
from datasets import TimeDataset, MiniDataset, FeatureDataset
import pickle
import importlib

dataset = MiniDataset(TimeDataset(), 5000000) # 0000
processed_dataset = FeatureDataset(dataset)
with open('processed_dataset.pkl', 'wb') as file:
    pickle.dump(processed_dataset, file)

KeyboardInterrupt: 

In [2]:
import pickle
dataset = pickle.load(open('processed_dataset.pkl', 'rb'))

In [3]:
dataset[0][0].shape

torch.Size([16384])

In [4]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score
import numpy

In [5]:
from tqdm.notebook import tqdm
def pytorch2sklearn(dataset):
    X = []
    Y = []
    for row in tqdm(dataset):
        # print(row)
        X.append(row[0].numpy()) # [el.item() for el in list(row[:-1])])
        Y.append(row[-1])
    return numpy.array(X), Y

X, y = pytorch2sklearn(dataset)
del dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [6]:
with open('X.pkl', 'wb') as file:
    pickle.dump(X, file)
with open('y.pkl', 'wb') as file:
    pickle.dump(y, file)

In [ ]:
clf = svm.SVC(verbose=1)
clf.fit(X_train, y_train)
predicted = clf.predict(X_test, y_test)
print(confusion_matrix(y_test, predicted, normalize='true'))
print(f1_score(y_test, predicted))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[LibSVM]